ukazkove zpracovani v 0414190301


In [1]:
idir="/home/data/sandbox/3C273_all/"

import os
ilist=os.listdir(idir)
from glob import glob


ele=[glob(idir+w+"/odf/*TAR") for w in ilist]
def ru1(w):
   os.chdir(idir+w+"/odf/");os.system("tar xzf "+w+".tar.gz")


In [23]:
qru1=lambda q,w:q.put(ru1(w))
todo=[ilist[e] for e in range(len(ele)) if len(ele[e])==0 and not os.path.exists(idir+ilist[e]+"/odf/ccf.cif")]
todo
jobs=[]
for i in todo:
    p = mp.Process(target=ru1, args=(i,))
    jobs.append(p)
    p.start()
jobs

[<Process(Process-25, started)>,
 <Process(Process-26, started)>,
 <Process(Process-27, started)>,
 <Process(Process-28, started)>,
 <Process(Process-29, started)>,
 <Process(Process-30, started)>,
 <Process(Process-31, started)>,
 <Process(Process-32, started)>,
 <Process(Process-33, started)>,
 <Process(Process-34, started)>,
 <Process(Process-35, started)>,
 <Process(Process-36, started)>,
 <Process(Process-37, started)>,
 <Process(Process-38, started)>]

In [2]:
import subprocess as sp
import multiprocessing as mp

In [35]:
ele=[glob(idir+w+"/odf/*TAR") for w in ilist]
def ru2(w):
   os.chdir(os.path.dirname(w))
   os.system("tar xf "+os.path.basename(w))
   #os.mkdir("pn")
   os.unlink(w)

ele2=glob(idir+"0*/odf/*SUM.ASC")
def ru3(u):
  os.environ["SAS_ODF"]=u
  print(sp.Popen("cifbuild",cwd=os.path.dirname(u),stdout=sp.PIPE).stdout.readlines()[-1])

In [29]:
# next step of unpacking
todo=[e[0] for e in ele if len(e)>0]
jobs=[]
for i in todo:
    p = mp.Process(target=ru2, args=(i,))
    jobs.append(p)
    p.start()
jobs

[<Process(Process-39, started)>,
 <Process(Process-40, started)>,
 <Process(Process-41, started)>,
 <Process(Process-42, started)>,
 <Process(Process-43, started)>,
 <Process(Process-44, started)>,
 <Process(Process-45, started)>,
 <Process(Process-46, started)>,
 <Process(Process-47, started)>,
 <Process(Process-48, started)>,
 <Process(Process-49, started)>,
 <Process(Process-50, started)>,
 <Process(Process-51, started)>,
 <Process(Process-52, started)>]

In [47]:
todo=[e for e in ele2 if not os.path.exists(os.path.dirname(e)+"/ccf.cif")]
jobs=[]
for i in todo[:10]:
    p = mp.Process(target=ru3, args=(i,))
    jobs.append(p)
    p.start()
jobs

[<Process(Process-73, started)>,
 <Process(Process-74, started)>,
 <Process(Process-75, started)>,
 <Process(Process-76, started)>,
 <Process(Process-77, started)>,
 <Process(Process-78, started)>]

In [51]:
ele3=glob(idir+"0*/odf/ccf.cif")
len(ilist),len(ele2),len(ele3)

(38, 38, 38)

In [3]:
os.environ.update({"HEADAS":"/isdc/heasoft-6.16/x86_64-unknown-linux-gnu-libc2.12/"})
scrp=os.popen("/isdc/heasoft-6.16/x86_64-unknown-linux-gnu-libc2.12/BUILD_DIR/headas-setup csh").read().strip()
scomm=open(scrp).readlines()
os.unlink(scrp)
#scomm
slist=[a.strip().replace('"','').split()[1:] for a in scomm]
os.environ.update(dict(slist))

In [4]:
os.environ["SAS_DIR"]="/isdc/xmmsas_20141104_1833"
rdir=os.environ["SAS_DIR"]
os.environ.update({'SAS_PATH':rdir,'SAS_VERBOSITY':'4','SAS_IMAGEVIEWER':'ds9','SAS_BROWSER':'firefox','SAS_SUPPRESS_WARNING':'1'})
os.environ['LD_LIBRARY_PATH']=rdir+'/lib:'+rdir+'/libextra:'+os.environ.get('LD_LIBRARY_PATH','.')
#+'/opt/rh/python33/root/usr/lib64:/isdc/heasoft-6.16/x86_64-unknown-linux-gnu-libc2.12/lib'
os.environ['LIBRARY_PATH']=rdir+'/libsys:'+rdir+'/libextra:'+rdir+'/lib'
os.environ['PATH']=rdir+'/binextra:'+rdir+'/bin:'+rdir+'/bin/devel:'+os.environ.get('PATH','')

In [53]:
sas_init=". "+os.environ["SAS_DIR"]+"/sas-setup.sh; "
def ru4(u):
  os.environ["SAS_ODF"]=os.path.dirname(u)
  os.environ["SAS_CCF"]=u
  print(sp.Popen("odfingest",cwd=os.path.dirname(u),stdout=sp.PIPE).stdout.readlines()[-1])
  return
#ru4(ele3[4])

In [56]:
ele4=glob(idir+"0*/odf/*SAS")
dir4=[os.path.dirname(e) for e in ele4]
todo=[e for e in ele3 if os.path.dirname(e) not in dir4]
print(len(todo),todo[:5])
jobs=[]
for i in todo[:8]:
    p = mp.Process(target=ru4, args=(i,))
    jobs.append(p)
    p.start()

5 ['/home/data/sandbox/3C273_all/0414190101/odf/ccf.cif', '/home/data/sandbox/3C273_all/0414191001/odf/ccf.cif', '/home/data/sandbox/3C273_all/0725060601/odf/ccf.cif', '/home/data/sandbox/3C273_all/0725060501/odf/ccf.cif', '/home/data/sandbox/3C273_all/0725060301/odf/ccf.cif']


In [58]:
sele4 = [os.path.dirname(e) for e in ele4 if not os.path.exists(os.path.dirname(e)+'/mos')]
def ru5(u,mode='pn'):
  os.environ["SAS_ODF"]=u
  os.environ["SAS_CCF"]=u+'/ccf.cif'
  os.mkdir(u+'/'+mode)
  print(sp.Popen("e"+mode[0]+"proc",cwd=u+'/'+mode,stdout=sp.PIPE,stderr=sp.PIPE).stdout.readlines()[-1])
  return

jobs=[]
for i in sele4[:14]:
    p = mp.Process(target=ru5, args=(i,'mos'),name=i[-14:-4])
    jobs.append(p)
    p.start()
jobs

b'odfingest:- odfingest (odfingest-3.30)  [xmmsas_20141104_1833-14.0.0] ended:    2015-03-17T16:48:28.000\n'


[<Process(0414190401, started)>,
 <Process(0725060401, started)>,
 <Process(0414190501, started)>,
 <Process(0414190601, started)>,
 <Process(0414190701, started)>,
 <Process(0414190201, started)>,
 <Process(0414190801, started)>,
 <Process(0725060701, started)>]

In [60]:
zz=[q for q in jobs if q.is_alive()]
print(len(zz))
zz

7
b'emproc:- emproc (epicproc-2.17.4)  [xmmsas_20141104_1833-14.0.0] ended:    2015-03-17T16:49:24.000\n'


[<Process(0414190401, started)>,
 <Process(0725060401, started)>,
 <Process(0414190501, started)>,
 <Process(0414190601, started)>,
 <Process(0414190701, started)>,
 <Process(0414190201, started)>,
 <Process(0414190801, started)>]

In [19]:
res1=glob(idir+"0*/odf/pn/*ingEvts.ds")
res2=glob(idir+"0*/odf/mos/*ingEvts.ds")

[(os.path.basename(r),os.path.getsize(r)//2**20) for r in res2]

b'emproc:- emproc (epicproc-2.17.4)  [xmmsas_20141104_1833-14.0.0] ended:    2015-03-17T16:02:48.000\n'
b'emproc:- emproc (epicproc-2.17.4)  [xmmsas_20141104_1833-14.0.0] ended:    2015-03-17T16:02:51.000\n'
b'emproc:- emproc (epicproc-2.17.4)  [xmmsas_20141104_1833-14.0.0] ended:    2015-03-17T16:02:55.000\n'
b'emproc:- emproc (epicproc-2.17.4)  [xmmsas_20141104_1833-14.0.0] ended:    2015-03-17T16:03:01.000\n'
b'emproc:- emproc (epicproc-2.17.4)  [xmmsas_20141104_1833-14.0.0] ended:    2015-03-17T16:03:02.000\n'


[('0382_0137551001_EMOS2_U008_ImagingEvts.ds', 1),
 ('0382_0137551001_EMOS2_U005_ImagingEvts.ds', 3),
 ('0382_0137551001_EMOS1_U004_ImagingEvts.ds', 1),
 ('0382_0137551001_EMOS1_U008_TimingEvts.ds', 3),
 ('0382_0137551001_EMOS1_U007_ImagingEvts.ds', 5),
 ('0382_0137551001_EMOS1_U008_ImagingEvts.ds', 1),
 ('0382_0137551001_EMOS2_U008_TimingEvts.ds', 3),
 ('0382_0137551001_EMOS2_U007_ImagingEvts.ds', 1),
 ('0382_0137551001_EMOS2_U004_ImagingEvts.ds', 1),
 ('0094_0126700201_EMOS1_S001_ImagingEvts.ds', 25),
 ('0094_0126700201_EMOS2_S002_ImagingEvts.ds', 24),
 ('0655_0159960101_EMOS2_S004_ImagingEvts.ds', 47),
 ('0655_0159960101_EMOS1_S003_ImagingEvts.ds', 47),
 ('1023_0136551001_EMOS1_S001_ImagingEvts.ds', 17),
 ('1023_0136551001_EMOS2_S002_ImagingEvts.ds', 18),
 ('0472_0112770601_EMOS2_S003_ImagingEvts.ds', 1),
 ('0472_0112770601_EMOS1_S002_TimingEvts.ds', 0),
 ('0472_0112770601_EMOS1_U002_ImagingEvts.ds', 1),
 ('0472_0112770601_EMOS1_U002_TimingEvts.ds', 1),
 ('0472_0112770601_EMOS1_S002